# Excel Splitter with Enhanced SharePoint Integration

This notebook provides an enhanced interface for splitting Excel files by reviewer with direct SharePoint integration using Microsoft Graph API.

## Key Features
- 📂 Creates application-specific folder structure
- 📊 Splits Excel by reviewer with filtered views
- 🔐 Direct SharePoint API integration with SSO
- 📧 Automatic email lookup from Microsoft 365
- ✉️ Email notifications to reviewers
- ☑️ Interactive reviewer selection with checkboxes

## Prerequisites
- Python 3.9 or higher
- Microsoft 365 account with SharePoint access
- Required packages: pandas, openpyxl, ipywidgets, msal, requests

## Step 1: Install Required Packages

In [ ]:
# Install required packages
import sys
import subprocess

required_packages = [
    'pandas',
    'openpyxl', 
    'ipywidgets',
    'msal',
    'requests'
]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("✓ All required packages installed")

## Step 2: Import Libraries and Setup Authentication

In [ ]:
import os
import sys
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import json
import requests
import msal
import webbrowser
from urllib.parse import quote
import time
from typing import Dict, List, Optional, Tuple

# Handle tkinter import
try:
    import tkinter as tk
    from tkinter import filedialog
    TKINTER_AVAILABLE = True
except ImportError:
    TKINTER_AVAILABLE = False

print("✓ Libraries imported successfully")

# Microsoft Graph API Configuration
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'
SHAREPOINT_API_VERSION = 'v1.0'

# Required scopes for our operations
SCOPES = [
    'User.Read',
    'User.ReadBasic.All', 
    'Sites.ReadWrite.All',
    'Mail.Send'
]

# Global variables for authentication
access_token = None
msal_app = None
accounts = None

## Step 3: Authentication Functions

In [ ]:
def get_auth_config():
    """Get authentication configuration"""
    # These would typically come from environment variables or a config file
    # For now, we'll use placeholders that the user can fill in
    return {
        'client_id': '',  # Your Azure AD app client ID
        'tenant_id': '',  # Your Azure AD tenant ID
        'redirect_uri': 'http://localhost:8400'  # Local redirect for auth
    }

def initialize_msal_app():
    """Initialize MSAL application"""
    global msal_app
    config = get_auth_config()
    
    if not config['client_id'] or not config['tenant_id']:
        print("⚠️ Please configure your Azure AD app credentials in the auth_config")
        return False
    
    authority = f"https://login.microsoftonline.com/{config['tenant_id']}"
    
    msal_app = msal.PublicClientApplication(
        config['client_id'],
        authority=authority
    )
    return True

def authenticate_interactive():
    """Authenticate using interactive browser flow"""
    global access_token, accounts
    
    if not msal_app:
        if not initialize_msal_app():
            return None
    
    # Check if we have cached accounts
    accounts = msal_app.get_accounts()
    
    if accounts:
        # Try to get token silently for the first account
        result = msal_app.acquire_token_silent(SCOPES, account=accounts[0])
        if result:
            access_token = result['access_token']
            print(f"✓ Authenticated as: {accounts[0]['username']}")
            return access_token
    
    # If silent auth fails, use interactive flow
    print("🔐 Opening browser for authentication...")
    result = msal_app.acquire_token_interactive(
        scopes=SCOPES,
        prompt="select_account"
    )
    
    if 'access_token' in result:
        access_token = result['access_token']
        print(f"✓ Successfully authenticated as: {result.get('account', {}).get('username', 'Unknown')}")
        return access_token
    else:
        print(f"❌ Authentication failed: {result.get('error_description', 'Unknown error')}")
        return None

def make_api_call(endpoint: str, method: str = 'GET', data: dict = None, headers: dict = None, retry_count: int = 3) -> dict:
    """Make authenticated API call to Microsoft Graph with retry logic"""
    if not access_token:
        print("❌ Not authenticated. Please authenticate first.")
        return None
    
    default_headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    
    if headers:
        default_headers.update(headers)
    
    for attempt in range(retry_count):
        try:
            # Set timeout for all requests
            timeout = 30  # seconds
            
            if method == 'GET':
                response = requests.get(endpoint, headers=default_headers, timeout=timeout)
            elif method == 'POST':
                response = requests.post(endpoint, headers=default_headers, json=data, timeout=timeout)
            elif method == 'PATCH':
                response = requests.patch(endpoint, headers=default_headers, json=data, timeout=timeout)
            else:
                raise ValueError(f"Unsupported method: {method}")
            
            if response.status_code == 200 or response.status_code == 201:
                return response.json() if response.content else {}
            elif response.status_code == 429:  # Rate limited
                retry_after = int(response.headers.get('Retry-After', 2 ** attempt))
                print(f"⏳ Rate limited. Waiting {retry_after} seconds...")
                time.sleep(retry_after)
                continue
            elif response.status_code == 401:  # Unauthorized
                print("❌ Authentication expired. Please re-authenticate.")
                return None
            else:
                print(f"❌ API call failed: {response.status_code} - {response.text[:200]}")
                if attempt < retry_count - 1:
                    wait_time = 2 ** attempt
                    print(f"⏳ Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    return None
                    
        except requests.exceptions.Timeout:
            print(f"⏱️ Request timeout (attempt {attempt + 1}/{retry_count})")
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
            else:
                return None
        except requests.exceptions.ConnectionError:
            print(f"🔌 Connection error (attempt {attempt + 1}/{retry_count})")
            if attempt < retry_count - 1:
                time.sleep(2 ** attempt)
            else:
                return None
        except Exception as e:
            print(f"❌ API call error: {e}")
            return None
    
    return None

print("✓ Authentication functions defined with retry logic")

## Step 4: SharePoint and Email Functions

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """Sanitize folder name for filesystem and SharePoint compatibility"""
    # Replace problematic characters
    replacements = {
        '/': '_',
        '\\': '_',
        ':': '_',
        '*': '_',
        '?': '_',
        '"': '_',
        '<': '_',
        '>': '_',
        '|': '_',
        '#': '_',
        '%': '_'
    }
    
    sanitized = name.strip()
    for char, replacement in replacements.items():
        sanitized = sanitized.replace(char, replacement)
    
    # Truncate if too long (max 255 chars)
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def validate_email(email: str) -> bool:
    """Basic email validation"""
    import re
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(pattern, email) is not None

def lookup_user_email(display_name: str) -> Optional[str]:
    """Lookup user email from Microsoft 365 by display name"""
    if not access_token:
        return None
    
    try:
        # Escape special characters in display name for filter
        escaped_name = display_name.replace("'", "''")
        
        # Search for users with matching display name
        search_filter = f"displayName eq '{escaped_name}' or startswith(displayName, '{escaped_name}')"
        endpoint = f"{GRAPH_API_ENDPOINT}/users?$filter={quote(search_filter)}&$select=displayName,mail,userPrincipalName"
        
        result = make_api_call(endpoint)
        
        if result and 'value' in result:
            users = result['value']
            if len(users) == 1:
                # Single match found
                email = users[0].get('mail') or users[0].get('userPrincipalName')
                return email if validate_email(email) else None
            elif len(users) > 1:
                # Multiple matches - try exact match
                for user in users:
                    if user.get('displayName', '').lower() == display_name.lower():
                        email = user.get('mail') or user.get('userPrincipalName')
                        return email if validate_email(email) else None
                # Return first match if no exact match
                email = users[0].get('mail') or users[0].get('userPrincipalName')
                return email if validate_email(email) else None
    except Exception as e:
        print(f"Error looking up email for {display_name}: {e}")
    
    return None

def get_sharepoint_site_id(site_url: str) -> Optional[str]:
    """Get SharePoint site ID from URL"""
    # Extract site path from full URL
    # Example: https://company.sharepoint.com/sites/teamsite -> sites/teamsite
    if 'sharepoint.com' in site_url:
        parts = site_url.split('sharepoint.com/')
        if len(parts) > 1:
            site_path = parts[1].strip('/')
            
            # Get site ID
            endpoint = f"{GRAPH_API_ENDPOINT}/sites/root:/{site_path}"
            result = make_api_call(endpoint)
            
            if result and 'id' in result:
                return result['id']
    
    return None

def share_folder_with_user(site_id: str, folder_path: str, user_email: str, permission: str = 'write') -> bool:
    """Share a folder with a specific user"""
    # URL encode the folder path for SharePoint
    encoded_path = quote(folder_path, safe='')
    
    # Create sharing invitation
    endpoint = f"{GRAPH_API_ENDPOINT}/sites/{site_id}/drive/root:/{encoded_path}:/invite"
    
    data = {
        "requireSignIn": True,
        "sendInvitation": True,
        "roles": [permission],
        "recipients": [
            {
                "email": user_email
            }
        ],
        "message": "You have been granted access to review files in this folder."
    }
    
    result = make_api_call(endpoint, method='POST', data=data)
    return result is not None

def send_notification_email(to_email: str, reviewer_name: str, folder_name: str, site_url: str) -> bool:
    """Send email notification to reviewer"""
    endpoint = f"{GRAPH_API_ENDPOINT}/me/sendMail"
    
    # Escape HTML entities in names
    import html
    safe_reviewer_name = html.escape(reviewer_name)
    safe_folder_name = html.escape(folder_name)
    
    email_body = f"""
    <html>
    <body>
        <p>Dear {safe_reviewer_name},</p>
        
        <p>You have been granted access to review files in the following SharePoint folder:</p>
        
        <p><strong>Folder:</strong> {safe_folder_name}<br>
        <strong>Location:</strong> <a href="{site_url}">{site_url}</a></p>
        
        <p>The folder contains filtered data specific to your review assignments. 
        Any changes you make will be synchronized with the master file through SharePoint's co-authoring feature.</p>
        
        <p>Please click the link above to access your files.</p>
        
        <p>Best regards,<br>
        Review Coordination Team</p>
    </body>
    </html>
    """
    
    data = {
        "message": {
            "subject": f"Access Granted: Review Files for {reviewer_name}",
            "body": {
                "contentType": "HTML",
                "content": email_body
            },
            "toRecipients": [
                {
                    "emailAddress": {
                        "address": to_email
                    }
                }
            ]
        },
        "saveToSentItems": True
    }
    
    result = make_api_call(endpoint, method='POST', data=data)
    return result is not None

print("✓ SharePoint and email functions defined with enhanced validation")

## Step 5: File Processing Functions (from original notebook)

In [ ]:
# Copy all the helper functions from the original notebook
def find_column(worksheet, column_name):
    """Find column index by name"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"Cannot find '{column_name}' column! Please check column name")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """Copy selected document types to destination"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def copy_specific_files(file_list, dest_dir, copy_word=True, copy_pdf=True, copy_all=False):
    """Copy specific files from a list to destination"""
    copied_files = []
    
    for file_path in file_list:
        if not os.path.isfile(file_path):
            continue
            
        file_ext = os.path.splitext(file_path)[1].lower()
        
        should_copy = False
        
        if copy_all:
            should_copy = True
        else:
            if copy_word and file_ext in ['.doc', '.docx']:
                should_copy = True
            elif copy_pdf and file_ext == '.pdf':
                should_copy = True
        
        if should_copy:
            dest_path = os.path.join(dest_dir, os.path.basename(file_path))
            shutil.copy2(file_path, dest_path)
            copied_files.append(os.path.basename(file_path))
    
    return copied_files

def copy_all_files_from_folder(source_dir, dest_dir, exclude_excel=True):
    """Copy all files from source directory to destination"""
    copied_files = []
    
    for file in os.listdir(source_dir):
        file_path = os.path.join(source_dir, file)
        if os.path.isfile(file_path):
            if exclude_excel and file.lower().endswith(('.xlsx', '.xls')):
                continue
            
            dest_path = os.path.join(dest_dir, file)
            shutil.copy2(file_path, dest_path)
            copied_files.append(file)
    
    return copied_files

print("✓ File processing functions defined")

## Step 6: Enhanced UI Components

In [ ]:
# File selection functions with folder memory
last_excel_folder = os.path.expanduser("~")
last_docs_folder = os.path.expanduser("~")

# Global state for reviewer data
reviewer_data = {}  # Will store: {reviewer_name: {'email': str, 'selected': bool, 'status': str}}
sharepoint_site_url = None
sharepoint_site_id = None

def select_excel_file():
    """Open file dialog to select Excel file"""
    global last_excel_folder
    if not TKINTER_AVAILABLE:
        print("❌ File dialog not available. Please type the file path manually.")
        return
        
    try:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        
        file_path = filedialog.askopenfilename(
            title="Select Excel File",
            filetypes=[
                ("Excel files", "*.xlsx *.xls"),
                ("All files", "*.*")
            ],
            initialdir=last_excel_folder
        )
        
        root.destroy()
        
        if file_path:
            excel_file.value = file_path
            last_excel_folder = os.path.dirname(file_path)
            print(f"✓ Selected: {os.path.basename(file_path)}")
        
    except Exception as e:
        print(f"❌ Error selecting file: {e}")

# Authentication UI
auth_status = widgets.HTML(value="<b>Status:</b> Not authenticated")
auth_button = widgets.Button(
    description='Authenticate',
    layout=widgets.Layout(width='150px')
)

# Azure AD Configuration inputs
client_id_input = widgets.Text(
    value='',
    placeholder='Enter Azure AD Client ID',
    description='Client ID:',
    layout=widgets.Layout(width='400px')
)

tenant_id_input = widgets.Text(
    value='',
    placeholder='Enter Azure AD Tenant ID',
    description='Tenant ID:',
    layout=widgets.Layout(width='400px')
)

# SharePoint configuration
sharepoint_url_input = widgets.Text(
    value='',
    placeholder='https://company.sharepoint.com/sites/teamsite',
    description='SharePoint Site:',
    layout=widgets.Layout(width='500px')
)

# File selection UI
excel_file = widgets.Text(
    value='',
    placeholder='Select Excel file...',
    description='Excel File:',
    layout=widgets.Layout(width='450px')
)

excel_browse_button = widgets.Button(
    description='Browse...',
    layout=widgets.Layout(width='100px'),
    disabled=not TKINTER_AVAILABLE
)

reviewer_column = widgets.Text(
    value='Reviewer',
    description='Reviewer Column:',
    layout=widgets.Layout(width='300px')
)

# Process button - avoid button_style for compatibility
process_button = widgets.Button(
    description='Split Excel File',
    layout=widgets.Layout(width='200px', height='40px')
)

# SharePoint sharing UI container (will be populated after processing)
sharing_container = widgets.VBox()
output = widgets.Output()

def authenticate_click(button):
    """Handle authentication button click"""
    with output:
        clear_output(wait=True)
        
        # Update auth config with input values
        config = get_auth_config()
        config['client_id'] = client_id_input.value.strip()
        config['tenant_id'] = tenant_id_input.value.strip()
        
        # Store config (in real app, this would be saved securely)
        global msal_app
        msal_app = None  # Reset to force reinitialization
        
        token = authenticate_interactive()
        if token:
            auth_status.value = "<b>Status:</b> <span style='color:green'>✓ Authenticated</span>"
            auth_button.description = "Re-authenticate"
        else:
            auth_status.value = "<b>Status:</b> <span style='color:red'>✗ Authentication failed</span>"

# Connect button events
auth_button.on_click(authenticate_click)
excel_browse_button.on_click(lambda x: select_excel_file())

# Display UI
display(HTML("<h3>1. Azure AD Configuration</h3>"))
display(client_id_input)
display(tenant_id_input)
display(widgets.HBox([auth_button, auth_status]))

display(HTML("<h3>2. SharePoint Configuration</h3>"))
display(sharepoint_url_input)

display(HTML("<h3>3. File Selection</h3>"))
display(widgets.HBox([excel_file, excel_browse_button]))
display(reviewer_column)

display(HTML("<br>"))
display(process_button)
display(output)
display(sharing_container)

print("✓ UI components ready (widget compatibility enhanced)")

## Step 7: Enhanced Processing Function with SharePoint Integration

In [ ]:
def create_reviewer_sharing_ui(reviewers: List[str]):
    """Create the reviewer selection UI with checkboxes"""
    global reviewer_data
    
    # Initialize reviewer data
    reviewer_data = {}
    
    # Create widgets
    select_all = widgets.Checkbox(
        value=True,
        description='Select All',
        indent=False,
        layout=widgets.Layout(width='150px')
    )
    
    reviewer_widgets = []
    
    # Lookup emails for all reviewers
    with output:
        print("\n🔍 Looking up reviewer emails...")
    
    for reviewer in reviewers:
        reviewer_str = str(reviewer).strip()
        
        # Create checkbox
        checkbox = widgets.Checkbox(
            value=True,
            description='',
            indent=False,
            layout=widgets.Layout(width='30px')
        )
        
        # Create name label
        name_label = widgets.Label(
            value=reviewer_str,
            layout=widgets.Layout(width='150px')
        )
        
        # Lookup email
        email = lookup_user_email(reviewer_str) if access_token else None
        
        # Create email input
        email_input = widgets.Text(
            value=email or '',
            placeholder='Enter email address',
            layout=widgets.Layout(width='250px')
        )
        
        # Create status label
        if email:
            status_html = "<span style='color:green'>✓ Found</span>"
        else:
            status_html = "<span style='color:orange'>⚠ Not found</span>"
        
        status_label = widgets.HTML(
            value=status_html,
            layout=widgets.Layout(width='100px')
        )
        
        # Store data
        reviewer_data[reviewer_str] = {
            'checkbox': checkbox,
            'email_input': email_input,
            'status_label': status_label,
            'selected': True,
            'email': email,
            'status': 'ready'
        }
        
        # Create row
        row = widgets.HBox([
            checkbox,
            name_label,
            email_input,
            status_label
        ])
        
        reviewer_widgets.append(row)
    
    # Select all handler
    def on_select_all_change(change):
        for data in reviewer_data.values():
            data['checkbox'].value = change['new']
    
    select_all.observe(on_select_all_change, names='value')
    
    # Share button
    share_button = widgets.Button(
        description='Share to Selected Reviewers',
        button_style='primary',
        layout=widgets.Layout(width='250px', height='40px')
    )
    
    send_email_checkbox = widgets.Checkbox(
        value=True,
        description='Send email notifications',
        indent=False
    )
    
    # Progress bar
    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(reviewers),
        description='Progress:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    progress_label = widgets.Label(value='')
    
    # Status output
    status_output = widgets.Output()
    
    # Share button handler
    def on_share_click(button):
        with status_output:
            clear_output()
            share_folders_to_reviewers(progress, progress_label, send_email_checkbox.value)
    
    share_button.on_click(on_share_click)
    
    # Build UI
    sharing_ui = widgets.VBox([
        widgets.HTML("<h3>4. SharePoint Sharing</h3>"),
        widgets.HTML("<p>Select reviewers to share folders with:</p>"),
        widgets.HBox([select_all]),
        widgets.HTML("<hr>"),
        widgets.VBox(reviewer_widgets),
        widgets.HTML("<hr>"),
        send_email_checkbox,
        widgets.HBox([share_button]),
        widgets.HBox([progress, progress_label]),
        status_output
    ])
    
    return sharing_ui

def share_folders_to_reviewers(progress_widget, progress_label, send_emails=True):
    """Share folders with selected reviewers"""
    global sharepoint_site_id
    
    # Get SharePoint site ID if not already fetched
    if not sharepoint_site_id:
        site_url = sharepoint_url_input.value.strip()
        if not site_url:
            print("❌ Please enter SharePoint site URL")
            return
        
        print(f"🔍 Getting SharePoint site ID...")
        sharepoint_site_id = get_sharepoint_site_id(site_url)
        
        if not sharepoint_site_id:
            print("❌ Failed to get SharePoint site ID. Please check the URL.")
            return
    
    # Get selected reviewers
    selected_reviewers = []
    for reviewer, data in reviewer_data.items():
        if data['checkbox'].value:
            email = data['email_input'].value.strip()
            if email:
                selected_reviewers.append((reviewer, email))
            else:
                print(f"⚠️ Skipping {reviewer} - no email address")
    
    if not selected_reviewers:
        print("❌ No reviewers selected or no email addresses provided")
        return
    
    print(f"\n📤 Sharing folders with {len(selected_reviewers)} reviewers...\n")
    
    # Reset progress
    progress_widget.value = 0
    progress_widget.max = len(selected_reviewers)
    
    success_count = 0
    
    for i, (reviewer, email) in enumerate(selected_reviewers):
        progress_label.value = f"{reviewer}"
        
        # Update status
        data = reviewer_data[reviewer]
        data['status_label'].value = "<span style='color:blue'>⏳ Sharing...</span>"
        
        try:
            # Share folder
            folder_path = reviewer  # Assuming folder name is reviewer name
            success = share_folder_with_user(sharepoint_site_id, folder_path, email)
            
            if success:
                data['status_label'].value = "<span style='color:green'>✓ Shared</span>"
                print(f"✓ {reviewer}: Folder shared")
                
                # Send email if requested
                if send_emails:
                    email_sent = send_notification_email(
                        email, 
                        reviewer, 
                        folder_path,
                        sharepoint_url_input.value.strip()
                    )
                    if email_sent:
                        data['status_label'].value = "<span style='color:green'>✓ Shared & Notified</span>"
                        print(f"  ✉️ Email notification sent")
                    else:
                        print(f"  ⚠️ Email notification failed")
                
                success_count += 1
            else:
                data['status_label'].value = "<span style='color:red'>✗ Failed</span>"
                print(f"✗ {reviewer}: Sharing failed")
                
        except Exception as e:
            data['status_label'].value = "<span style='color:red'>✗ Error</span>"
            print(f"✗ {reviewer}: Error - {str(e)}")
        
        # Update progress
        progress_widget.value = i + 1
        time.sleep(0.5)  # Small delay to avoid rate limiting
    
    # Final summary
    progress_label.value = "Complete!"
    print(f"\n✅ Sharing complete! Successfully shared with {success_count}/{len(selected_reviewers)} reviewers.")

def process_excel(button):
    """Main processing function"""
    with output:
        clear_output()
        
        # Validate inputs
        if not excel_file.value:
            print("❌ Please select an Excel file")
            return
        
        if not access_token:
            print("❌ Please authenticate first")
            return
        
        file_path = excel_file.value.strip()
        column = reviewer_column.value.strip()
        
        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            return
        
        print(f"📁 Processing: {os.path.basename(file_path)}")
        print(f"📊 Reviewer Column: {column}")
        print("=" * 50)
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column not in df.columns:
                print(f"❌ Column '{column}' not found")
                print(f"Available columns: {', '.join(df.columns)}")
                return
            
            # Get unique reviewers
            reviewers = df[column].dropna().unique().tolist()
            print(f"✓ Found {len(reviewers)} reviewers")
            
            # Process each reviewer (create folders and split files)
            base_dir = os.path.dirname(file_path)
            base_name = os.path.basename(file_path)
            name_without_ext = os.path.splitext(base_name)[0]
            ext = os.path.splitext(base_name)[1]
            
            processed = 0
            
            for reviewer in reviewers:
                reviewer_name = str(reviewer).strip()
                print(f"\n📝 Processing: {reviewer_name}")
                
                # Create reviewer folder
                reviewer_folder = os.path.join(base_dir, reviewer_name)
                os.makedirs(reviewer_folder, exist_ok=True)
                
                # Create filtered Excel
                new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
                dst_path = os.path.join(reviewer_folder, new_filename)
                wb = load_workbook(file_path)
                ws = wb.active
                
                try:
                    # Find column and apply filter
                    col_idx = find_column(ws, column)
                    max_row = ws.max_row
                    max_col = ws.max_column
                    
                    filter_range = f"A1:{get_column_letter(max_col)}{max_row}"
                    ws.auto_filter.ref = filter_range
                    ws.auto_filter.add_filter_column(col_idx - 1, [reviewer_name])
                    
                    # Hide non-matching rows
                    for row in range(2, max_row + 1):
                        cell_value = ws.cell(row=row, column=col_idx).value
                        if cell_value != reviewer_name:
                            ws.row_dimensions[row].hidden = True
                    
                    wb.save(dst_path)
                    print(f"  ✓ Created: {new_filename}")
                    processed += 1
                    
                except Exception as e:
                    print(f"  ❌ Error: {e}")
                finally:
                    wb.close()
            
            # Summary
            print("\n" + "=" * 50)
            print(f"✅ PROCESSING COMPLETE!")
            print(f"📊 Processed {processed}/{len(reviewers)} reviewers")
            print(f"📁 Output location: {base_dir}")
            
            # Create sharing UI
            global sharing_container
            sharing_ui = create_reviewer_sharing_ui(reviewers)
            sharing_container.children = [sharing_ui]
            
        except Exception as e:
            print(f"\n❌ Fatal error: {e}")
            import traceback
            traceback.print_exc()

# Attach handler
process_button.on_click(process_excel)
print("✓ Processing function ready")

## Instructions for Use

### 1. Azure AD Setup
1. Register an app in Azure AD
2. Grant the following API permissions:
   - User.Read
   - User.ReadBasic.All
   - Sites.ReadWrite.All
   - Mail.Send
3. Configure redirect URI: `http://localhost:8400`
4. Copy the Client ID and Tenant ID

### 2. Using the Notebook
1. Enter your Azure AD credentials and authenticate
2. Enter your SharePoint site URL
3. Select your Excel file and process it
4. Review the email addresses found for each reviewer
5. Select which reviewers to share with
6. Click "Share to Selected Reviewers"

### 3. What Happens
- Folders are created for each reviewer
- Excel files are filtered and saved
- SharePoint permissions are set via API
- Email notifications are sent (if enabled)
- Progress is shown in real-time

### Troubleshooting
- **Authentication fails**: Check Client ID and Tenant ID
- **Email not found**: Manually enter the email address
- **Sharing fails**: Ensure you have permissions to share on the SharePoint site
- **No folders created**: Check Excel file path and reviewer column name